In [22]:
!pip install sentencepiece
!pip install --upgrade pip
!pip install pandas
!pip install datasets

In [23]:
import re
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
from google.colab import drive

In [24]:
import torch
device = torch.device("cuda")

In [25]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
# Function to handle whitespace in text
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

# Load the csebuetnlp/xlsum dataset from Hugging Face
xlsum_dataset = load_dataset("csebuetnlp/xlsum","bengali", split="test")

# Assuming the 'article' is nested within a 'document' key
articles = xlsum_dataset['text']

# Initialize the tokenizer and model
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.to(device)
# Create a list to store the summaries
summaries = []

for article_text in articles:
    article_text = WHITESPACE_HANDLER(article_text)

   # Move input tensor to the same device as the model
    input_ids = tokenizer(
        [article_text],
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=512
    )["input_ids"].to(device)

    output_ids = model.generate(
        input_ids=input_ids,
        min_length=70,
        max_length=90,
        no_repeat_ngram_size=2,
        num_beams=4
    )[0]

    summary = tokenizer.decode(
        output_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )

    summaries.append(summary)

# Create a DataFrame with the summaries
result_df = pd.DataFrame({"Article": xlsum_dataset['text'], "Summary": summaries})
drive_path = '/content/drive/My Drive/Colab Notebooks/'
# Save the DataFrame to a CSV file
result_df.to_csv(os.path.join(drive_path, "summaries.csv"), index=False, encoding='utf-8')

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [27]:
result_df.to_csv(os.path.join(drive_path, "summaries1.csv"), index=False, encoding = 'utf-8-sig')


In [28]:
result_df.to_csv(os.path.join(drive_path, "summaries2.csv"), index=False, encoding = 'utf-16')
